In [1]:
# Import tensorflow 
import tensorflow as tf
# import keras tensorflow layer for architecture; Dense, Flatten and Conv2D
from tensorflow.keras.layers import Dense, Flatten, Conv2D
#import tensorflow keras for Model (Sequential)
from tensorflow.keras import Model

In [2]:
# Load and Prepare the Dataset
mnist = tf.keras.datasets.mnist

In [3]:
#loaded data
mnist

<module 'tensorflow_core.keras.datasets.mnist' from 'C:\\Users\\uib43225\\.conda\\envs\\ROSE\\lib\\site-packages\\tensorflow_core\\python\\keras\\api\\_v2\\keras\\datasets\\mnist\\__init__.py'>

In [4]:
# SPilit the data into train and test 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)

(60000, 28, 28)


In [5]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
x_train = x_train[..., tf.newaxis].astype("float32")
print(x_train.shape)
x_test = x_test[..., tf.newaxis].astype("float32")

(60000, 28, 28, 1)


In [7]:
# use tf.data to batch and shuffle the dataset; to make ssame dimension length of data
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

In [8]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [9]:
train_ds

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.uint8)>

In [10]:
# BUild Model using tensorflow

In [11]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

In [12]:
model = MyModel()

In [13]:
model

In [14]:
# choose an optimiser and loss function for training
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [15]:
optimizer = tf.keras.optimizers.Adam()

In [16]:
# select matrics to measure the loss and accuracy
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [17]:
# use tf.GradientTape(automatic differentiation) to train the model

def train_step(images, labels):
    with tf.GradientTape() as Tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        
    gradients = Tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

In [18]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
Epochs = 5

for epoch in range(Epochs):
    # In each Training steps, calculating the loss, and accuracy and then reset for next epoch of training steps 
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
      
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100))

Epoch 1, Loss: 0.135336235165596, Accuracy: 95.961669921875, Test Loss: 0.07257215678691864, Test Accuracy: 97.5199966430664
Epoch 2, Loss: 0.041687775403261185, Accuracy: 98.67333221435547, Test Loss: 0.06505770981311798, Test Accuracy: 98.11000061035156
Epoch 3, Loss: 0.019642429426312447, Accuracy: 99.39500427246094, Test Loss: 0.06217449903488159, Test Accuracy: 98.25
